# Environment Explore

In [196]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack
import numpy as np
import mediapy as media
import jax.numpy as jp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:
params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_21_1_250_clip_0.p",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

# Creating env objects

In [198]:
env = RodentSingleClipTrack(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitatio

In [199]:
key = random.PRNGKey(0)
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)

In [200]:
type(env._ref_traj)

mocap_preprocess.ReferenceClip

In [201]:
n = env.reset(key)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

shape of each of the reference trajectory

In [202]:
jax.tree_util.tree_map(
    lambda x: print(x.shape),
    env._ref_traj,
)

(250, 3)
(250, 5, 3)
(250, 18, 3)
(250, 18, 4)
(250, 3)
(250, 4, 3)
(250, 67)
(250, 67)
(0,)
(250, 3)
(250, 4)
(0,)
(250, 3)


ReferenceClip(angular_velocity=None, appendages=None, body_positions=None, body_quaternions=None, center_of_mass=None, end_effectors=None, joints=None, joints_velocity=None, markers=None, position=None, quaternion=None, scaling=None, velocity=None)

In [203]:
stacked = jp.hstack([env._ref_traj.position,
           env._ref_traj.quaternion,
           env._ref_traj.joints,
           ])

stacked[:,env.joint_order].shape

(250, 30)

# Creating state objects & reset_fn, get_obs_fn
`reset_fn` and `get_obs_fn` is called when initilizing the environment and creating the state object

In [204]:
rng = random.PRNGKey(0)
state = env.reset(rng)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

In [205]:
state.pipeline_state.xpos[:,0] # all x values
state.pipeline_state.xpos[env.body_idxs].shape

(18, 3)

In [206]:
env._ref_traj.body_positions.shape

(250, 18, 3)

reference function check

In [207]:
env._ref_traj.joints_velocity[0,:] - state.pipeline_state.qvel[6:]

Array([ 1.1143157e+00, -4.4521580e+00, -3.0700223e+00, -4.4953046e+00,
        6.9346321e-01, -4.7531295e+00,  3.7333816e-01, -8.9423358e-02,
        8.3647041e+00,  1.6094770e+01,  1.4872874e+01,  0.0000000e+00,
        3.3132515e+00, -1.6146489e+00, -8.5274601e-01, -1.2136062e+01,
       -5.0887170e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  2.6349113e+00,  3.0387416e+00,
        1.8484621e+00, -1.1493291e+01, -3.5080731e+00, -9.8731136e+00,
       -3.4217072e-01,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00, -5.0105155e-05,  2.5493584e+01,  0.0000000e+00,
      

end effector position

In [208]:
state.pipeline_state.xpos[jp.array(env._end_eff_idx)]

Array([[0.24422027, 0.34589922, 0.00972859],
       [0.18785599, 0.33485827, 0.03198195],
       [0.30204108, 0.32896948, 0.01415975],
       [0.3096328 , 0.28116018, 0.01297225]], dtype=float32)

In [209]:
state.obs

Array([ 0.28308997,  0.31726265,  0.07212678, ..., -0.30106932,
       -0.01390321,  0.03926991], dtype=float32)

In [210]:
jp.isclose(state.obs, 0)

Array([False, False, False, ..., False, False, False], dtype=bool)

In [211]:
jp.isclose(state.obs, 0).sum()/state.obs.shape[0]

Array(0.1521739, dtype=float32)

# Stepping Env: Test step_fn, reward_fn, & termination_fn
`step_fn`, which calls `reward_fn` & `termination_fn` is only used when actually making action, not when setting up environment and state object

In [29]:
# next = jit_reset(key)
# print("starting rollout")
# mu = 0
# sigma = .1
# rollout = []
# for _ in range(500):
#     next = jit_step(next, mu + sigma * random.normal(key, shape=(env.sys.nu,)))
#     rollout.append(next)

In [30]:
# import mediapy as media
# import os

# os.environ["MUJOCO_GL"] = "glfw"
# rollout_data = [s.pipeline_state for s in rollout]

# video = env.render(rollout_data, camera='side_alt', height=500, width=500)

# media.show_video(video, fps=1.0 / env.dt)

# Stepping Environment & Observe Changes in Frames

In [31]:
env = RodentSingleClipTrack(params)
reset_fn = jax.jit(jax.vmap(env.reset))
step_fn = jax.jit(jax.vmap(env.step))

n_envs = 1
key_envs = jax.random.split(key, n_envs)
env_state = reset_fn(key_envs)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitatio

KeyboardInterrupt: 

In [ ]:
# new_state = env_state
# check = []
# error = []
# for _ in range(1):
#     mu = 0
#     sigma = 0.3
#     action =  mu + sigma * random.normal(key, shape=(n_envs, env.sys.nu))
#     new_state = step_fn(new_state, action)
#     check.append(new_state.info['step_after_reset'])
#     error.append(new_state.info['termination_error'])

# print(check, error)

In [ ]:
# new_state = step_fn(new_state, action)
# check_state = reset_fn(key_envs)
# check_state = step_fn(check_state, action)
# check_state.info['episode_frame'] # does reset correctly

# Test Wrapper For Environment

In [ ]:
from brax import envs
wrap_for_training = envs.training.wrap

wrap_env = wrap_for_training(
    env,
    episode_length=1000,
    action_repeat=1
  )

In [ ]:
wrap_env.env.env.env

# Comparing Relative Functions
Instantiate `dm_contol` MultiClipMocapTracking Environment with ur ref_traj data, seems a bit challenging, using rodent trajectory is not applicable probably (observation tell).

In [ ]:
env_state.obs

Array([[-0.00060661,  0.4388396 ,  0.07299647, ...,  0.04394839,
         0.43269745,  0.01097612]], dtype=float32)

In [ ]:
from walker import Rat
from dm_control.locomotion.walkers import rescale

walker = Rat()
rescale.rescale_subtree(walker.mjcf_model, 0.9, 0.9)

In [ ]:
# from dm_control import composer
# from dm_control.locomotion import arenas
# from dm_control.locomotion import walkers
# from dm_control.locomotion.mocap import props
# from dm_control.locomotion.tasks.reference_pose import tracking
# from dm_control.locomotion.tasks.reference_pose import types
# from dm_control.utils import io as resources
# import os
# import h5py

# # Define the path to your motion capture data file
# file_name = 'all_snips_250.h5'
# current_directory = os.getcwd()
# TEST_FILE_PATH = os.path.join(current_directory, file_name)

# with h5py.File(TEST_FILE_PATH, 'r') as f:
#     dataset_keys = tuple(f.keys())
#     dataset = types.ClipCollection(ids=dataset_keys,)

# # Set up the mocap tracking task
# task = tracking.MultiClipMocapTracking(
#     walker=walker, #walkers.CMUHumanoidPositionControlled,
#     arena=arenas.Floor(),
#     ref_path=resources.GetResourceFilename(TEST_FILE_PATH),
#     ref_steps=(1, 2, 3, 4, 5),
#     min_steps=1,
#     dataset=dataset,
#     reward_type='comic',
# )

# # Initialize the environment
# env = composer.Environment(task=task)
# env.reset()